# NORMALIZACIÓN DEL DATASET

Queremos que los resultados por parte del llm que sean distintos a los que se le mencionaron en el prompt se organicen en una categoría llamada "inconsistente"

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('./base_con_normalizacion.xlsx')

In [ ]:
data.columns 

El prompt 9 tuvo la particularidad de que asignó múltiples estiquetas a un mismo comentario, en este sentido, se creo una nueva columna que se llama **respuesta_prompt9_procesado** donde se le asigno a estos valores una doble categoría

In [ ]:
# drop  respuesta_prompt9
data = data.drop(['respuesta_prompt9'], axis=1)

In [ ]:
data.head(3)

In [ ]:
data["respuesta_prompt9_procesado"].unique()

generación de vectores para prueba chi cuadrado en Wolfram:

In [ ]:
import numpy as np

# Asignar el DataFrame a una variable
df = prompts

# Crear 10 vectores con tamaño 100000
diccionario_map = {'POSITIVO': 1, 'NEGATIVO': 2, 'NEUTRAL': 3, 'INCONSISTENTE': 4}

vectores = {}
for i in range(9, 11):
    vector_nombre = f'X{i}'
    vectores[vector_nombre] = df[f'prompt {i}'].replace(diccionario_map).values

# Guardar los vectores en un archivo txt
with open('vectores2.txt', 'w') as file:
    for nombre, vector in vectores.items():
        file.write(f"{nombre} = {vector.tolist()}\n")

In [ ]:
# Guardar los vectores pares en un archivo txt
with open('vectores_pares.txt', 'w') as file_pares:
    for i in range(2, 9, 2):
        vector_nombre = f'X{i}'
        file_pares.write(f"{vector_nombre} = {{{', '.join(map(str, vectores[vector_nombre]))}}};\n")

# Guardar los vectores impares en un archivo txt
with open('vectores_impares.txt', 'w') as file_impares:
    for i in range(1, 9, 2):
        vector_nombre = f'X{i}'
        file_impares.write(f"{vector_nombre} = {{{', '.join(map(str, vectores[vector_nombre]))}}};\n")



In [ ]:
df = prompts
# drop Texto Comentario
df = df.drop(['Texto Comentario'], axis=1)

# Visualización de coincidencias y clasificaciones

In [ ]:
# Calcular los totales por fila
totales = df[['POSITIVO', 'NEUTRAL', 'NEGATIVO', 'INCONSISTENTE']].sum(axis=1)

# Crear una nueva tabla con el formato "conteo (porcentaje)"
contingency_table_total = pd.DataFrame({
    'POSITIVO': df['POSITIVO'].map(lambda x: f"{x} ({(x / totales.sum() * 100):.2f}%)"),
    'NEUTRAL': df['NEUTRAL'].map(lambda x: f"{x} ({(x / totales.sum() * 100):.2f}%)"),
    'NEGATIVO': df['NEGATIVO'].map(lambda x: f"{x} ({(x / totales.sum() * 100):.2f}%)"),
}, index=df.index)

# Imprimir la tabla con los conteos y porcentajes
print(contingency_table_total)


In [ ]:
# Supongamos que tienes tus datos cargados en un DataFrame llamado 'df'
contingency_table_total = pd.DataFrame({
    'POSITIVO': (df == 'POSITIVO').sum(),
    'NEUTRAL': (df == 'NEUTRAL').sum(),
    'NEGATIVO': (df == 'NEGATIVO').sum(),

})

print(contingency_table_total)

In [ ]:
# Calcular el total por columna (prompt)
totales_por_prompt = df.apply(lambda col: col.value_counts())

# Calcular los porcentajes por columna (prompt)
porcentajes_por_prompt = totales_por_prompt.div(totales_por_prompt.sum(axis=0), axis=1) * 100

# Combinar las cantidades y porcentajes en una tabla formateada
contingency_table_total = totales_por_prompt.fillna(0).astype(int).astype(str) + " (" + porcentajes_por_prompt.fillna(0).round(2).astype(str) + "%)"

# Reemplazar NaN por vacío (si alguna clasificación no está en un prompt)
contingency_table_total = contingency_table_total.fillna("0 (0.00%)")

# transponer la tabla
contingency_table_total = contingency_table_total.T

contingency_table_total 


# Visualización de inconsistencias entre respuestas de los prompts

In [ ]:
# filtrar los inconsistentes
prompts = prompts[~prompts.isin(['INCONSISTENCIA'])]

In [ ]:
# Filtra las filas donde al menos un valor en las columnas (excepto la columna 'No') es diferente
filtered_prompts = prompts[prompts.drop(columns=['No']).apply(lambda row: len(set(row)) > 1, axis=1)]

# Muestra el DataFrame filtrado
filtered_prompts

In [ ]:
# numero de filas de filtered_prompts

len(filtered_prompts)

# Matriz de proporción de coincidencias

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
# todas las respuestas que coincidan hacer una matriz de la proporción de coincidencias
# para cada par de columnas
import numpy as np

coincidencias = np.zeros((prompts.shape[1], prompts.shape[1]))

for i in range(prompts.shape[1]):
    for j in range(prompts.shape[1]):
        coincidencias[i, j] = (prompts.iloc[:, i] == prompts.iloc[:, j]).mean()

coincidencias = pd.DataFrame(coincidencias, columns=prompts.columns, index=prompts.columns)

coincidencias

In [ ]:
coincidencias.to_csv('./results/matrices/coincidencias_matrix.csv', index=False)

# Análisis de embeddings

In [ ]:
import os 
import openai
from openai import OpenAI

In [ ]:
api_key = "API KEY"

In [ ]:
openai.api_key = os.getenv(api_key)

In [ ]:
client = OpenAI(
    api_key=api_key,
)

In [ ]:
# create a function that takes a string and returns the embedding with comparacion["prompt"]

def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

# get the embeddings for each prompt
comparacion["prompt_embedding"] = comparacion["Prompt"].apply(get_embedding)

In [ ]:
comparacion.columns

In [ ]:
comparacion

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

In [ ]:
embeddings = np.array(comparacion["prompt_embedding"].tolist())

In [ ]:
embeddings

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
# make embeddings_pca a DataFrame an enumerate the columns with the prompts 
embeddings_pca = pd.DataFrame(embeddings_pca, columns=["PCA1", "PCA2"])

embeddings_pca["# "] = comparacion["# "]

embeddings_pca

In [ ]:
# beautify the plot Prompt Similarity in PCA Space 
plt.figure()
plt.scatter(embeddings_pca["PCA1"], embeddings_pca["PCA2"])

for i, txt in enumerate(embeddings_pca["# "]):
    plt.annotate(txt, (embeddings_pca["PCA1"][i], embeddings_pca["PCA2"][i]))

plt.title("Prompt Similarity in PCA Space")
plt.xlabel("PCA1")
plt.ylabel("PCA2")

plt.show()

In [ ]:
embeddings_pca

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
# varianza explicada
pca.explained_variance_ratio_

In [ ]:
import plotly.graph_objects as go

# Creamos el scatter plot 3D
fig = go.Figure(data=[go.Scatter3d(
    x=embeddings_pca[:, 0],
    y=embeddings_pca[:, 1],
    z=embeddings_pca[:, 2],
    mode='markers+text',
    text=comparacion["# "],  # Añade los textos a los puntos
    marker=dict(
        size=5,
        color='blue',
        opacity=0.8
    )
)])

# Ajustamos el diseño
fig.update_layout(
    scene = dict(
        xaxis_title='Componente 1',
        yaxis_title='Componente 2',
        zaxis_title='Componente 3'
    ),
    title='Scatter Plot 3D de Embeddings'
)

# Mostramos el gráfico interactivo
fig.show()


Varianza acomulada de 71

# Cálculo de correlación entre embeddings

In [ ]:
# calcular la correlación de los embeddings y graficar la matriz de correlación
correlacion = np.corrcoef(embeddings)
correlacion = pd.DataFrame(correlacion, columns=comparacion["# "], index=comparacion["# "])

In [ ]:
correlacion

In [ ]:
correlacion.to_csv('./results/matrices/correlacion_matrix.csv', index=False)

In [ ]:
# hacer la matriz de distancia euclidiana entre los embeddings
from scipy.spatial.distance import pdist, squareform

distancias = pdist(embeddings)

distancias = squareform(distancias)

distancias = pd.DataFrame(distancias, columns=comparacion["# "], index=comparacion["# "])

distancias

# Kapa Kohen

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score
import numpy as np

def codificar_columnas(df):
    """
    Codifica todas las columnas categóricas en números utilizando LabelEncoder.
    """
    df = df.copy()
    le = LabelEncoder()
    
    for column in df.columns:
        df[column] = le.fit_transform(df[column].astype(str))  # Convertimos a string antes de codificar
    
    return df

In [ ]:
def calcular_kappa(df, columnas_excluir=None):
    """
    Esta función calcula el coeficiente Kappa de Cohen para cada par de columnas.
    
    Args:
    df (DataFrame): El DataFrame que contiene las respuestas de los evaluadores.
    columnas_excluir (list): Lista de columnas a excluir del análisis, como la columna 'No' que representa IDs.
    
    Returns:
    DataFrame: Un DataFrame con los coeficientes Kappa para cada par de columnas.
    """
    # Excluir las columnas indicadas en columnas_excluir
    if columnas_excluir:
        df = df.drop(columns=columnas_excluir, errors='ignore')
    
    # Codificar las columnas categóricas (si es necesario)
    df = codificar_columnas(df)
    
    columnas = df.columns  # Las columnas restantes después de excluir 'No' u otras columnas
    kappa_matriz = pd.DataFrame(np.zeros((len(columnas), len(columnas))), index=columnas, columns=columnas)
    
    # Iterar sobre todos los pares posibles de columnas
    for i in range(len(columnas)):
        for j in range(i + 1, len(columnas)):
            col1 = columnas[i]
            col2 = columnas[j]
            
            # Calcular el coeficiente Kappa de Cohen entre las dos columnas (evaluadores)
            kappa = cohen_kappa_score(df[col1], df[col2])
            
            # Almacenar el valor de Kappa en la matriz simétrica
            kappa_matriz.at[col1, col2] = kappa
            kappa_matriz.at[col2, col1] = kappa  # La matriz es simétrica
    
    return kappa_matriz

# Excluir la columna 'No', que representa IDs
columnas_a_excluir = ['No']

# Calcular solo el Kappa para todos los pares de prompts excluyendo 'No'
kappa_matriz = calcular_kappa(prompts, columnas_a_excluir)

# Mostrar la matriz de Kappa
kappa_matriz



In [ ]:
# imprimir una matriz con los resultados de Kappa
kappa_matrix = pd.DataFrame(index=prompts.columns, columns=prompts.columns)

In [ ]:
import pandas as pd
import numpy as np

def calcular_kappa_sin_paquete(df, columnas_excluir=None):
    """
    Calcula el coeficiente Kappa de Cohen entre todas las columnas del DataFrame sin usar paquetes externos.
    
    Args:
    df (DataFrame): El DataFrame que contiene las respuestas de los evaluadores.
    columnas_excluir (list): Lista de columnas a excluir del análisis, como la columna 'No' que representa IDs.
    
    Returns:
    DataFrame: Un DataFrame con los coeficientes Kappa para cada par de columnas.
    """
    # Excluir las columnas indicadas en columnas_excluir
    if columnas_excluir:
        df = df.drop(columns=columnas_excluir, errors='ignore')
    
    columnas = df.columns  # Las columnas restantes después de excluir las indicadas
    kappa_matriz = pd.DataFrame(np.zeros((len(columnas), len(columnas))), index=columnas, columns=columnas)
    
    # Función auxiliar para calcular Kappa entre dos columnas (sin paquetes)
    def calcular_kappa_col(col1, col2):
        n = len(col1)
        acuerdo_observado = sum([1 for i in range(n) if col1[i] == col2[i]]) / n
        
        unique_values = set(col1).union(set(col2))
        frec_col1 = {val: col1.count(val) / n for val in unique_values}
        frec_col2 = {val: col2.count(val) / n for val in unique_values}
        
        acuerdo_esperado = sum([frec_col1[val] * frec_col2[val] for val in unique_values])
        
        if acuerdo_esperado == 1:
            return 1  # Si el acuerdo esperado es perfecto, el Kappa es 1
        
        kappa = (acuerdo_observado - acuerdo_esperado) / (1 - acuerdo_esperado)
        return kappa
    
    # Iterar sobre todos los pares posibles de columnas
    for i in range(len(columnas)):
        for j in range(i, len(columnas)):  # Comparar columna consigo misma y con otras
            if i == j:
                kappa_matriz.iloc[i, j] = 1  # El Kappa de una columna consigo misma es 1
            else:
                col1 = df[columnas[i]].tolist()
                col2 = df[columnas[j]].tolist()
                kappa = calcular_kappa_col(col1, col2)
                kappa_matriz.iloc[i, j] = kappa
                kappa_matriz.iloc[j, i] = kappa  # La matriz es simétrica
    
    return kappa_matriz

# Excluir la columna 'No', que representa IDs
columnas_a_excluir = ['No']

# Calcular solo el Kappa para todos los pares de prompts excluyendo 'No'
kappa_matriz = calcular_kappa_sin_paquete(prompts, columnas_a_excluir)

# Mostrar la matriz de Kappa
kappa_matriz



!pwd  # Esto muestra el directorio actual

In [ ]:
!pwd  # Esto muestra el directorio actual


In [ ]:
kappa_matriz.to_csv('./results/matrices/kappa_matrix.csv', index=False)

# Matriz de proporciones

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
# todas las respuestas que coincidan hacer una matriz de la proporción de coincidencias
# para cada par de columnas
import numpy as np

coincidencias = np.zeros((prompts.shape[1], prompts.shape[1]))

for i in range(prompts.shape[1]):
    for j in range(prompts.shape[1]):
        coincidencias[i, j] = (prompts.iloc[:, i] == prompts.iloc[:, j]).mean()

coincidencias = pd.DataFrame(coincidencias, columns=prompts.columns, index=prompts.columns)

coincidencias

In [ ]:
coincidencias.to_csv('./results/matrices/coincidencias_matrix.csv', index=False)